In [21]:
import pandas as pd
import pickle
from gensim.models import Phrases
import re

In [22]:
file_type = 'thread'
field_name = 'title'

In [23]:
pickle_file = open(f'{file_type}_{field_name}_token_lists.pickle', "rb" )
corpus = pickle.load( pickle_file )
pickle_file.close()

In [24]:
clean_corpus = list()
for document in corpus:
    clean_document = list()
    for word in document:
        if len(word) > 1:
            clean_document.append(re.sub(r"[\?|!|\.|,|:|_|\(|\)]", '', word))
    clean_corpus.append(clean_document)
corpus = clean_corpus

In [53]:
min_count = 25
threshold = 50
fix_parameter_values = False

In [54]:
def find_value_in_nested_loop(ngram,prev_gram_models,sent,n):
    assert n > 1
    value = None
    for i in range(n-2):
        n_gram_model = prev_gram_models[i]
        if i == 0:
            value = n_gram_model[sent]
        else:
            value = n_gram_model[value]
    return value

In [55]:
def get_phrases_ngram(corpus, min_count, threshold,fix_parameter_values=fix_parameter_values):
    has_more_pharses = True
    n = 2
    prev_gram = []
    all_phrases = set()
    while has_more_pharses:
        curr_phrases = set()
        if n == 2:
            ngram = Phrases(corpus, min_count=min_count, threshold=threshold)
        else:
            ngram = Phrases(find_value_in_nested_loop(ngram,prev_gram,corpus,n), min_count=min_count, threshold=threshold)
        for sent in corpus:
            if n == 2:
                curr_phrases = curr_phrases.union({b for b in ngram[sent] if b.count('_') == 1})
            else:
                curr_phrases = curr_phrases.union({t for t in find_value_in_nested_loop(ngram,prev_gram,sent,n) if t.count('_') == n})
        n += 1
        prev_gram.append(ngram)
        all_phrases = all_phrases.union(curr_phrases)
        has_more_pharses = len(curr_phrases) > 0
        if not fix_parameter_values:
            min_count = max(min_count/2,1)
            threshold = max(threshold/2,1)
    print(f'the biggest n is {n-1}')
    return all_phrases

In [56]:
phrases = get_phrases_ngram(corpus, min_count, threshold, fix_parameter_values)
f = open('phrases.txt','w')
for phrase in phrases:
    f.write(f'{phrase}\n')
f.close()

the biggest n is 3
